In [2]:
import h5py
import numpy as np
import k3d
import trimesh
import os
from omegaconf import OmegaConf
from airexo.helpers.urdf_robot import forward_kinematic
from airexo.helpers.constants import ROBOT_PREDEFINED_TRANSFORMATION

# 1. 读取关节数据
scene_path = "/data/haoxiang/data/airexo2/task_0013/train/scene_0001/lowdim"

with h5py.File(f"{scene_path}/robot_left.h5", 'r') as f:
    left_joint = f['joint_pos'][0]  # 第一帧，shape=(7,)

with h5py.File(f"{scene_path}/robot_right.h5", 'r') as f:
    right_joint = f['joint_pos'][0]

with h5py.File(f"{scene_path}/gripper_left.h5", 'r') as f:
    left_gripper = f['gripper_width'][0] if 'gripper_width' in f else 0.05

with h5py.File(f"{scene_path}/gripper_right.h5", 'r') as f:
    right_gripper = f['gripper_width'][0] if 'gripper_width' in f else 0.05

# 拼接关节+夹爪
left_joint = np.concatenate([left_joint, [left_gripper]])
right_joint = np.concatenate([right_joint, [right_gripper]])

# 2. 加载 joint_cfgs
left_cfg = OmegaConf.load("airexo/configs/joint/left/robot.yaml")
right_cfg = OmegaConf.load("airexo/configs/joint/right/robot.yaml")

# 3. 前向运动学
cur_transforms, visuals_map = forward_kinematic(
    left_joint=left_joint,
    right_joint=right_joint,
    left_joint_cfgs=left_cfg,
    right_joint_cfgs=right_cfg,
    is_rad=True, 
    urdf_file="airexo/urdf_models/robot/robot_inhand.urdf",
    with_visuals_map=True
)

# 4. 加载并变换 mesh
plot = k3d.plot()
for link, transform in cur_transforms.items():
    if link not in visuals_map: continue
    
    for v in visuals_map[link]:
        if v.geom_param is None: continue
        
        # 构建完整路径
        mesh_path = os.path.join("airexo/urdf_models/robot", v.geom_param)
        
        if not os.path.exists(mesh_path):
            print(f"Warning: Mesh file not found: {mesh_path}")
            continue

        # --- 核心修改部分 ---
        # 使用 force='mesh' 确保加载进来的是 Trimesh 对象而不是 Scene
        mesh = trimesh.load(mesh_path, force='mesh')
        
        # 应用变换矩阵
        tf = ROBOT_PREDEFINED_TRANSFORMATION @ transform.matrix() @ v.offset.matrix()
        mesh.apply_transform(tf)
        
        # 渲染到 k3d
        # 建议加上颜色以区分不同部件（可选）
        plot += k3d.mesh(mesh.vertices.astype(np.float32), 
                         mesh.faces.astype(np.uint32),
                         color=0xd3d3d3) # 浅灰色

plot.display()

Output()